In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
import cv2
import mediapipe as mp
import time
frameWidth = 640
frameHeight = 480
cap = cv2.VideoCapture(0)
cap.set(3, frameWidth)
cap.set(4, frameHeight)
cap.set(10, 150)

Draw_landmark = mp.solutions.drawing_utils
Result = mp.solutions.hands
hand = Result.Hands()

tip=[4,8,12,16,20]
while True:
    success, img = cap.read(0)
    RGB_img=cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    output=hand.process(RGB_img)
    hand_type="none"
    hand_count=0
    if output.multi_hand_landmarks:
        hand_count=len(output.multi_handedness)
        for hand_multi in output.multi_handedness:
            if(hand_multi.classification[0].label=="Left"):
                hand_type="Right"
            elif(hand_multi.classification[0].label=="Right"):
                hand_type="Left"

        for lms in output.multi_hand_landmarks:
            list = [[], []]
            for id,lm in enumerate(lms.landmark):
                #print(id, lm)

                height,width,channel=img.shape
                x,y= int(lm.x*width) , int(lm.y*height)
                #print(id,x,y)
                list[0].append(x)
                list[1].append(y)
            finger_list = []
            if(hand_type=="Right"):
                if(list[0][tip[0]]>list[0][tip[0]-1]):
                    finger_list.append(1)
                else:
                    finger_list.append(0)
            elif(hand_type=="Left"):
                if (list[0][tip[0]] > list[0][tip[0] - 1]):
                    finger_list.append(0)
                else:
                    finger_list.append(1)
            for count in range(1,5):
                if(list[1][tip[count]]<list[1][tip[count]-2]):
                    finger_list.append(1)
                else:
                    finger_list.append(0)
            Draw_landmark.draw_landmarks(img,lms,Result.HAND_CONNECTIONS)
            if(hand_count<2):
                print(finger_list)
            else:
                print("Multiple Hands detected")

    cv2.imshow("Result",img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break




In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))

  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))